In [8]:
with open('input.txt') as f:
    real_data = f.read()

In [9]:
import aocd
import dataclasses
import numpy as np
import enum

test_data = """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]"""

In [24]:
from typing import Sequence, Union, Type, Tuple
import itertools


def in_order(left, right, debug=False) -> Union[bool, None]:
    """See if the pair is in order.

    Args:
        left: the first list.
        right: the second list.
        
    Returns:
        whether or not the lists are in order.
    """
    if debug: print(f"Compare {left} vs {right}")
    if left == right:
        return None
    
    if type(left) == int:
        if type(right) == int:
            return left < right
        return in_order([left], right)
    elif type(right) == int:
        return in_order(left, [right])
    else:
        # both are lists
        for i_left, i_right in zip(left, right):
            if debug: print(f"Compare {i_left} vs {i_right}")
            is_in_order = in_order(i_left, i_right)
            if is_in_order is None:
                continue
            else:
                return is_in_order
        return in_order(len(left), len(right))

        
@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    Args:
        raw_data: the raw input data.
    """
    raw_data: str
        
    def __post_init__(self):
        pairs = self.raw_data.split("\n\n")
        parsed_pairs = []
        for pair in pairs:
            list0, list1 = pair.split("\n")
            parsed_pairs.append([eval(list0), eval(list1)])
        self.pairs = parsed_pairs

    def debug(self) -> None:
        """For debugging."""
        for idx, pair in enumerate(self.pairs):
            print(f"== Pair {idx + 1} ==")
            pair_in_order = in_order(*pair)
            print(f"{'In order' if pair_in_order else 'Not in order'}")
            in_order(*pair, debug=True)
            print("")
            
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """        
        total_score = 0
        for idx, pair in enumerate(self.pairs):
            pair_in_order = in_order(*pair)
            total_score += (idx + 1) if pair_in_order else 0
        return total_score

In [25]:
SolverA(test_data).debug()

== Pair 1 ==
In order
Compare [1, 1, 3, 1, 1] vs [1, 1, 5, 1, 1]
Compare 1 vs 1
Compare 1 vs 1
Compare 3 vs 5

== Pair 2 ==
In order
Compare [[1], [2, 3, 4]] vs [[1], 4]
Compare [1] vs [1]
Compare [2, 3, 4] vs 4

== Pair 3 ==
Not in order
Compare [9] vs [[8, 7, 6]]
Compare 9 vs [8, 7, 6]

== Pair 4 ==
In order
Compare [[4, 4], 4, 4] vs [[4, 4], 4, 4, 4]
Compare [4, 4] vs [4, 4]
Compare 4 vs 4
Compare 4 vs 4

== Pair 5 ==
Not in order
Compare [7, 7, 7, 7] vs [7, 7, 7]
Compare 7 vs 7
Compare 7 vs 7
Compare 7 vs 7

== Pair 6 ==
In order
Compare [] vs [3]

== Pair 7 ==
Not in order
Compare [[[]]] vs [[]]
Compare [[]] vs []

== Pair 8 ==
Not in order
Compare [1, [2, [3, [4, [5, 6, 7]]]], 8, 9] vs [1, [2, [3, [4, [5, 6, 0]]]], 8, 9]
Compare 1 vs 1
Compare [2, [3, [4, [5, 6, 7]]]] vs [2, [3, [4, [5, 6, 0]]]]



In [12]:
SolverA(test_data).find_answer()

13

In [13]:
SolverA(real_data).find_answer()

5196

In [27]:
from typing import Sequence, Union, Type, Tuple
import itertools

def in_order(left, right, debug=False) -> Union[bool, None]:
    """See if the pair is in order.

    Args:
        left: the first list.
        right: the second list.
        
    Returns:
        whether or not the lists are in order.
        If equal, then return None.
    """
    if debug: print(f"Compare {left} vs {right} -> ", end="")
    if left == right:
        return None
    
    if type(left) == int:
        if type(right) == int:
            return left < right
        return in_order([left], right)
    elif type(right) == int:
        return in_order(left, [right])
    else:
        # both are lists
        for i_left, i_right in zip(left, right):
            if debug: print(f"Compare {i_left} vs {i_right} -> ", end="")
            is_in_order = in_order(i_left, i_right)
            if is_in_order is None:
                continue
            else:
                if debug: print(f"{is_in_order}")
                return is_in_order
        return in_order(len(left), len(right))


        
def sort_lines(lines: Sequence) -> Sequence:
    """Uses bubble sort to sort the lines into the right order.
    
    Args:
        lines: the lists to sort.
        
    Returns:
        sorted lines.
    """
    n_lines = len(lines)
    for i in range(n_lines - 1) :
        swap = False
        
        for j in range(n_lines - 1) :
            if not in_order(lines[j], lines[j + 1]): 
                lines[j], lines[j + 1] = lines[j + 1], lines[j]
                swap = True
                
        if not swap:
            continue

    return lines
        
@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    Args:
        raw_data: the raw input data.
    """
    raw_data: str
        
    def __post_init__(self):
        pairs = self.raw_data.split("\n\n")
        parsed_pairs = []
        for pair in pairs:
            list0, list1 = pair.split("\n")
            parsed_pairs.append([eval(list0), eval(list1)])
        self.pairs = parsed_pairs
        
        self.lines = []
        for pair in self.pairs:
            self.lines.extend(pair)
        divider_packets = [[[2]], [[6]]]
        self.lines.extend(divider_packets)

    def debug(self) -> None:
        """For debugging."""
        sorted_lines = sort_lines(self.lines)
        keys = []
        for idx, line in enumerate(sorted_lines):
            if line == [[2]] or line == [[6]]:
                keys.append(idx + 1)
        
        return sorted_lines
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """     
        sorted_lines = sort_lines(self.lines)
        
        keys = []
        for idx, line in enumerate(sorted_lines):
            if line == [[2]] or line == [[6]]:
                keys.append(idx + 1)
        return keys[0] * keys[1]


In [23]:
SolverB(test_data).debug()

[[],
 [[]],
 [[[]]],
 [1, 1, 3, 1, 1],
 [1, 1, 5, 1, 1],
 [[1], [2, 3, 4]],
 [1, [2, [3, [4, [5, 6, 0]]]], 8, 9],
 [1, [2, [3, [4, [5, 6, 7]]]], 8, 9],
 [[1], 4],
 [[2]],
 [3],
 [[4, 4], 4, 4],
 [[4, 4], 4, 4, 4],
 [[6]],
 [7, 7, 7],
 [7, 7, 7, 7],
 [[8, 7, 6]],
 [9]]

In [28]:
SolverB(test_data).find_answer()

140

In [29]:
SolverB(real_data).find_answer()

22134